- a software system to bring out information of web

- smart search engines can be considered unsupervised learning approaches, due to the nature of clustering related information without such label in hand

- Search Engines have evolved **from a text input and output service to** an experience that cuts across voice, video, documents, and conversations


- an **infinite problem** to solve


- **related** to information retrieval, language understanding


- the **value that an effective search tool can bring to a business is enormous**; a key piece of intellectual property. Often a search bar is the main interface between customers and the business. 
    - create a competitive advantage by delivering an improved user experience.
    


search engine popular approaches:
- manual implementation with dataframe + tf-idf
- Elastic Search + BM25
- BM25 + Azure Cognitive Search

Requirements:
- Search index for storing each document, reflecting relevant information and up to date information
    - data can be reorganized by date (suggestion)
- Query understanding
    - takes sentence and preprocessed data information **directly without much context**
    - we can extract words or tokens from the query to match **article_type** (suggestion)
        - query to match tags (done)
    - we can filter the search by either blog or News (suggestion)
        - or add multiple results available (blog, News, or both)
    - BM25 + Azure Cognitive search
- Query ranking
    - by consine similarity

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
docs_df = pd.read_json('../Data/husna.json')

C:\Users\modaj\AppData\Local\Temp\ipykernel_230368\1371443659.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## 1- Library and Data Imports

In [3]:
import numpy as np
import pandas as pd
import time

# for text cleaning and preprocessing
import re
from nltk.corpus import stopwords
import string 
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
docs_df = pd.read_json('../Data/husna.json')

## 2- Data Preparation

#### 2.1 preparing data for cleaning

In [5]:
# MODIFIED
docs_df = docs_df.drop(columns=['publisher', 'crawled_at', 'url', 'published_at'], axis=1)

In [6]:
docs_df_dropped = docs_df.drop(index=
                               docs_df[(docs_df['content'].str.len() == 0) & (docs_df['title'] == '')].index, axis=0)
docs_df_dropped = docs_df_dropped.reset_index(drop=True)
docs_df = docs_df_dropped

In [7]:
docs_df['text'] = docs_df['content'].apply(lambda x: " ".join(x))

## 3- Data Cleaning

important data cleaning functions:
- remove punctuation
- tokenization 
- stem words

**cleaning functions not implemented**: removing repeating characters, stop words, emoji, hashtags

In [7]:
# REMOVE LATER
vectorizer = TfidfVectorizer()

In [8]:
def show_info_text(df_col):
    print(f"-> Number of Documents: {docs_df.shape[0]}")
    print('-' * 50, end='\n\n')

    print('-> Documents - First 150 letters')
    print()
    for i, document_i in enumerate(docs_df['text_clean'][:20]):
        print(f"Document Number {i+1}: {document_i[:150]}..")
        print()

    print('-' * 50)
    
def data_preprocessing(df_col):
    # Instantiate a TfidfVectorizer object
    vectorizer = TfidfVectorizer()
    
    # It fits the data and transform it as a vector
    X = vectorizer.fit_transform(df_col)
    # Convert the X as transposed matrix
    X = X.T.toarray()
    # Create a DataFrame and set the vocabulary as the index
    df = pd.DataFrame(X, index=vectorizer.get_feature_names())
    return df, vectorizer

### 3.1 data cleaning (ver.1)

handle:
- removing mentions
- removing punctuation
- removing Arabic diacritics (short vowels and other harakahs) 
    - حركات وشد
- removing elongation 
    - مد
- removing stopwords (which is available in NLTK corpus)
    - normal stopwords (not specific to arabic)
- remove words from languages other than arabic and english

In [75]:
docs_df_cleaned = docs_df.copy()

In [76]:
# punctuation symbols
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words = stopwords.words()
arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def clean_text(txt): 
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    txt = txt.translate(translator)
    
    # remove Tashkeel
    txt = re.sub(arabic_diacritics, '', txt)
    
    # remove longation
    txt = re.sub("[إأآا]", "ا", txt)
    txt = re.sub("ى", "ي", txt)
    txt = re.sub("ؤ", "ء", txt)
    txt = re.sub("ئ", "ء", txt)
    txt = re.sub("ة", "ه", txt)
    txt = re.sub("گ", "ك", txt)
    
    # remove stopwords
    txt = ' '.join(word for word in txt.split() if word not in stop_words)
    
    # remove non-arabic words, or non-numbers, or non-english words in the text
    txt = re.sub(r'[^a-zA-Z\s0-9\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD.0-9]+'
                 ,' ', txt)
    
    return txt

In [78]:
docs_df_cleaned = docs_df.drop(columns=['_id', 'title', 'summary', 'content', 'text'])

start_time = time.time()
docs_df_cleaned['text_clean'] = docs_df['text'].apply(clean_text)

# docs_df_cleaned['summary_clean'] = docs_df['summary'].apply(clean_text) # no need for now
docs_df_cleaned['title_clean'] = docs_df['title'].apply(clean_text)
text_clean_enc_df, clean1_vect = data_preprocessing(docs_df_cleaned['text_clean'])
time_measure = (time.time() - start_time) * 10**3

text_clean_enc_df 

C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\SEA_venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0,1,2,3,4,5,6,7,8,9,...,5074,5075,5076,5077,5078,5079,5080,5081,5082,5083
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00000015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ہر,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ہمارے,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ہولوکاسٹ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ہےکہ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
start_time = time.time()
text_clean_enc_df, clean1_vect = data_preprocessing(docs_df_cleaned['text_clean'])
# text_clean_enc_df = vectorizer.fit_transform(docs_df_cleaned2['text_clean'])
time_measure = (time.time() - start_time) * 10**3

print(f"preprocessing time taken: {time_measure}")

preprocessing time taken: 680.5415153503418


**NOTE** 
- ~6000 source documents -> ~5000 documents -> ~89,000 tokens
- Clean time for 'text' column: ~111 seconds
- **Problems**:
    - may not be normalized enough
    - words from other languages
    - confusing numbers (remove or keep?)
        - remove english numbers? or arabic numbers? or both?
        - should we remove words of letters mixed with numbers (E.g. COVID19)
    - links (remove or keep?)
        

### 3.1 data cleaning (ver.2)

handle:
- removing Arabic diacritics (short vowels and other harakahs)
- variation by form and spelling, based on context (Orthographic Ambiguity)
- existence of many forms for the same word (Morphological Richness)
- dialects (Dialectal Variation)
- different ways to write the same word when writing in dialectal Arabic, for which there is no agreed-upon standard
    - Orthographic Inconsistency
- removing longation and stop words
- remove words from languages other than arabic and english
   
these problems can possibly lead to immensly large vocabularies generated.

In [9]:
docs_df_cleaned2 = docs_df.copy()

In [10]:
# import the dediacritization tool
from camel_tools.utils.dediac import dediac_ar

# Reducing Orthographic Ambiguity
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar

# toknenization
from camel_tools.tokenizers.word import simple_word_tokenize

# Morphological Disambiguation (Maximum Likelihood Disambiguator)
from camel_tools.disambig.mle import MLEDisambiguator
mle = MLEDisambiguator.pretrained() # instantiation fo MLE disambiguator

# tokenization / lemmatization (choosing approach that best fit the project)
from camel_tools.tokenizers.morphological import MorphologicalTokenizer

import re
from nltk.corpus import stopwords

In [11]:
stop_word_list = pd.read_csv('../Data/stop_words/list.csv')['words'].to_list()
tokenizer = MorphologicalTokenizer(mle, scheme='atbtok', diac=False) # atbseg scheme 
def text_clean2(txt):
    # remove stopwords
    txt = ' '.join(word for word in txt.split() if word not in stop_word_list)
    
    # dediacritization
    txt = dediac_ar(txt)
    
    # normalization: Reduce Orthographic Ambiguity and Dialectal Variation
    txt = normalize_alef_maksura_ar(txt)
    txt = normalize_alef_ar(txt)
    txt = normalize_teh_marbuta_ar(txt)
    
    # normalization: Reducing Morphological Variation
    tokens = simple_word_tokenize(txt)
    disambig = mle.disambiguate(tokens)
    lemmas = [d.analyses[0].analysis['lex'] for d in disambig]
    tokens = tokenizer.tokenize(lemmas)
    txt = ' '.join(tokens)
    
    # remove longation
    txt = re.sub("[إأآا]", "ا", txt)
    txt = re.sub("ى", "ي", txt)
    txt = re.sub("ؤ", "ء", txt)
    txt = re.sub("ئ", "ء", txt)
    txt = re.sub("ة", "ه", txt)
    txt = re.sub("گ", "ك", txt)
    
    # remove non-arabic words, or non-numbers, or non-english words in the text
    txt = re.sub(r'[^a-zA-Z\s0-9\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD.0-9]+'
                 ,' ', txt)
    
    return txt

In [12]:
# apply to your text column
docs_df_cleaned2 = docs_df.drop(columns=['_id', 'title', 'summary', 'content', 'text'])
start_time = time.time()
docs_df_cleaned2['text_clean'] = docs_df['text'].apply(text_clean2)
time_measure = (time.time() - start_time) * 10**3

In [16]:
import spacy
from tqdm import tqdm

C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\SEA_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
docs_df_cleaned2['text_clean_vect'] = docs_df_cleaned2['text_clean'].apply(lambda x: x.split(' '))
docs_df_cleaned2

tags article_type  \
0     [التربية والتعليم, وزارة التربية والتعليم]      News          
1     [مكرمة أبناء المعلمين, وزارة التربية والتعليم]  News          
2     [الغاز الروسي, أوروبا, أزمة الطاقة]             News          
3     [التربية والتعليم, التوقيت الصيفي, المدارس]     News          
4     [وزارة العمل, المولد النبوي]                    News          
...                            ...                     ...          
5079  [الديموقراطية]                                  Blog          
5080  [ألوان الطيف السبعة, الشذوذ الجنسي]             Blog          
5081  [الإسلام, الإسلام السياسي]                      Blog          
5082  [تخفيضات]                                       Blog          
5083  []                                              News          

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [18]:
import spacy
from tqdm import tqdm

nlp = spacy.load("en_core_web_sm")
tok_text=[] # for our tokenised corpus

#Tokenising using SpaCy:
for doc in tqdm(nlp.pipe(docs_df_cleaned2.text_clean.values, disable=["tagger", "parser","ner"])):
    tok = [t.text for t in doc if t.is_alpha]
    tok_text.append(tok)

0it [00:00, ?it/s]C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\SEA_venv\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
5084it [00:52, 97.75it/s] 


In [38]:
# docs_df_cleaned2['text_clean_vect'].tolist()

In [48]:
from rank_bm25 import BM25Okapi
# bm25 = BM25Okapi(docs_df_cleaned2['text_clean_vect'].tolist())
bm25 = BM25Okapi(docs_df['text'].tolist())

In [51]:
query = "النبي"

# tokenized_query = query.lower().split(" ")
tokenized_query = query.split(" ")
# tokenized_query = query

t0 = time.time()
results = bm25.get_top_n(tokenized_query, docs_df.text.values, n=10)
t1 = time.time()
print(f'Searched records in {round(t1-t0,3) } seconds \n')
for i in results:
    print(i)
    print()

Searched records in 0.002 seconds 

تميل درجات الحرارة في عطلة نهاية الأسبوع للارتفاع الطفيف و تكون أعلى من معدلاتها السنوية في مثل هذا الوقت من العام بقليل. ويبقى الطقس صيفياً إعتيادياً في المرتفعات الجبلية، وحاراً نسبياً في بقية المناطق، ويكون الطقس حاراً في مناطق الأغوار والبحر الميت والعقبة و مناطق البادية. أما الرياح فتكون شمالية غربية معتدلة السرعة، وتنشط بعد الظهر مع احتمالية إثارة الغبار.  

كشف رئيس لجنة التعليم والشباب النيابية بلال المومني لـ  حسنى  اليوم الأحد أن وزير التعليم العالي محمد أبو قديس أكد للّجنة الخميس الماضي أن تغيير ست رؤساء جامعات مؤخرا جاء بناء على تقييم الأداء، خاصة فيما يتعلق بأداء الجامعات في تطوير نفسها في مسيرة التعليم عن بعد. ولفت المومني أن الوزير لم يقدم أسبابا محددة للجنة حول قرار تغيير الرؤساء، وأنه لم يخض في تفاصيل التقييم حرصا على القامات الأكاديمية التي تقود الجامعات، وأنه اكتفى بالتأكيد على أن التغيير جاء بناء على التقييم وليس قرارا عشوائيا. وكانت اللجنة عقدت اجتماعا الخميس الماضي بحضور وزير التربية وزير التعليم العالي لبحث عدد من القضايا على ر

In [ ]:
# text_clean_enc_df = data_preprocessing(docs_df_cleaned2['text_clean'])

# text_clean_enc_df, clean2_vect = data_preprocessing(docs_df_cleaned['text_clean'])
start_time = time.time()
# text_clean_enc_df, clean2_vect = data_preprocessing(docs_df_cleaned2['text_clean'])
# text_clean_enc_df = clean2_vect.fit_transform(docs_df_cleaned2['text_clean'])

time_measure = (time.time() - start_time) * 10**3

text_clean_enc_df

In [14]:
# fit + transform time

start_time = time.time()
text_clean_enc_df, clean2_vect = data_preprocessing(docs_df_cleaned2['text_clean'])
# text_clean_enc_df = clean2_vect.transform(docs_df_cleaned2['text_clean'])
time_measure = (time.time() - start_time) * 10**3

print('time measure: ', time_measure)
# text_clean_enc_df

time measure:  874.047040939331


C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\SEA_venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# using vectorizer time 


- ~6000 source documents -> ~5000 documents -> ~23,000 tokens
- Clean time for 'text' column: ~180 seconds
- Problems:
    - stopwords (remove or keep?)
    - normalization may have cut out too many tokens 
    - confusing numbers (remove or keep?)
        - remove english numbers? or arabic numbers? or both?
    - should we remove words of letters mixed with numbers (E.g. COVID19)
    - links (remove or keep?)
    
**NOTE** discuss with instructor before proceeding

### 3.x check results

In [396]:
# display(docs_df_cleaned.head())
i=0

In [515]:
# check results
print(f'--> {i}')
display(text_clean_enc_df.index[50*i:50*(i+1)])
display(text_clean_enc_df.index[-50*(i+1):(-50*i)-1])
i += 1

print('clean time: {:.2f} seconds'.format(time_measure * 10**-3))

--> 118


Index(['اجرامي', 'اجراميا', 'اجراميه', 'اجراه', 'اجراها', 'اجرت', 'اجرته',
       'اجرتها', 'اجره', 'اجرها', 'اجرهم', 'اجرو', 'اجروا', 'اجري', 'اجرياها',
       'اجرياهما', 'اجريت', 'اجرين', 'اجزاء', 'اجزاءه', 'اجزاءها', 'اجزم',
       'اجساد', 'اجسادهم', 'اجسام', 'اجساما', 'اجسامنا', 'اجسامهم', 'اجعلك',
       'اجل', 'اجلاء', 'اجلاءه', 'اجلاءها', 'اجلاءهم', 'اجلال', 'اجلت', 'اجلس',
       'اجله', 'اجلها', 'اجلهم', 'اجلي', 'اجماع', 'اجماعا', 'اجمال', 'اجمالي',
       'اجماليه', 'اجمع', 'اجمعت', 'اجمعوا', 'اجمعين'],
      dtype='object')

Index(['ومستوطنوه', 'ومستوطنيه', 'ومستوفيه', 'ومستوي', 'ومستويات', 'ومستوياته',
       'ومسجد', 'ومسجدها', 'ومسجل', 'ومسجله', 'ومسددين', 'ومسرحه', 'ومسرحيه',
       'ومسرحيين', 'ومسرعه', 'ومسري', 'ومسعر', 'ومسكين', 'ومسلح', 'ومسلحه',
       'ومسلسل', 'ومسلسلات', 'ومسلم', 'ومسلمات', 'ومسلمه', 'ومسلمين',
       'ومسلوقه', 'ومسمع', 'ومسميات', 'ومسنه', 'ومسنون', 'ومسوحات', 'ومسوده',
       'ومسور', 'ومسيء', 'ومسيحين', 'ومسيحيه', 'ومسيحيين', 'ومسيرات',
       'ومسيرته', 'ومسيرتهم', 'ومسيره', 'ومشابهه', 'ومشادات', 'ومشارفها',
       'ومشاركاتهم', 'ومشاركته', 'ومشاركتهم', 'ومشاركه'],
      dtype='object')

clean time: 99.80 seconds


In [ ]:
vocab_ = vectorizer.vocabulary_
print(f"number of unique words: {len(vocab_.keys())}")
most_freq_word = sorted(vocab_.items(), key=lambda x: x[1], reverse=True)[:1][0]
print('most frequent word is --> {} ({} times)'.format(most_freq_word[0], most_freq_word[1]))
score = len(vocab_.keys()) / most_freq_word[1]
print('Ratio: {:.3f}'.format(score))

## 4- Apply Cleaning on Query

In [74]:
stop_word_list

['،',
 'ء',
 'ءَ',
 'آ',
 'آب',
 'آذار',
 'آض',
 'آل',
 'آمينَ',
 'آناء',
 'آنفا',
 'آه',
 'آهاً',
 'آهٍ',
 'آهِ',
 'أ',
 'أبدا',
 'أبريل',
 'أبو',
 'أبٌ',
 'أجل',
 'أجمع',
 'أحد',
 'أخبر',
 'أخذ',
 'أخو',
 'أخٌ',
 'أربع',
 'أربعاء',
 'أربعة',
 'أربعمئة',
 'أربعمائة',
 'أرى',
 'أسكن',
 'أصبح',
 'أصلا',
 'أضحى',
 'أطعم',
 'أعطى',
 'أعلم',
 'أغسطس',
 'أفريل',
 'أفعل به',
 'أفٍّ',
 'أقبل',
 'أكتوبر',
 'أل',
 'ألا',
 'ألف',
 'ألفى',
 'أم',
 'أما',
 'أمام',
 'أمامك',
 'أمامكَ',
 'أمد',
 'أمس',
 'أمسى',
 'أمّا',
 'أن',
 'أنا',
 'أنبأ',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنتِ',
 'أنشأ',
 'أنه',
 'أنًّ',
 'أنّى',
 'أهلا',
 'أو',
 'أوت',
 'أوشك',
 'أول',
 'أولئك',
 'أولاء',
 'أولالك',
 'أوّهْ',
 'أى',
 'أي',
 'أيا',
 'أيار',
 'أيضا',
 'أيلول',
 'أين',
 'أيّ',
 'أيّان',
 'أُفٍّ',
 'ؤ',
 'إحدى',
 'إذ',
 'إذا',
 'إذاً',
 'إذما',
 'إذن',
 'إزاء',
 'إلى',
 'إلي',
 'إليكم',
 'إليكما',
 'إليكنّ',
 'إليكَ',
 'إلَيْكَ',
 'إلّا',
 'إمّا',
 'إن',
 'إنَّ',
 'إى',
 'إياك',
 'إياكم',
 'إياكما',
 'إياكن',


In [9]:
stop_word_list = pd.read_csv('../Data/stop_words/list.csv')['words'].to_list()
tokenizer = MorphologicalTokenizer(mle, scheme='atbtok', diac=False) # atbseg scheme 
def text_clean2_steps(txt):
    # remove stopwords
    txt = ' '.join(word for word in txt.split() if word not in stop_word_list)
    print('stopwords', txt)
    
    # dediacritization
    txt = dediac_ar(txt)
    print('dediacritization', txt)
    
    # normalization: Reduce Orthographic Ambiguity and Dialectal Variation
    txt = normalize_alef_maksura_ar(txt)
    print('Reduce Orthographic Ambiguity and Dialectal Variation', txt)
    txt = normalize_alef_ar(txt)
    print('Reduce Orthographic Ambiguity and Dialectal Variation', txt)
    txt = normalize_teh_marbuta_ar(txt)
    print('Reduce Orthographic Ambiguity and Dialectal Variation', txt)
    
    # normalization: Reducing Morphological Variation
    tokens = simple_word_tokenize(txt)
    disambig = mle.disambiguate(tokens)
    lemmas = [d.analyses[0].analysis['lex'] for d in disambig]
    tokens = tokenizer.tokenize(lemmas)
    txt = ' '.join(tokens)
    print('Reducing Morphological Variation', txt)
    
    # remove longation
    txt = re.sub("[إأآا]", "ا", txt)
    txt = re.sub("ى", "ي", txt)
    txt = re.sub("ؤ", "ء", txt)
    txt = re.sub("ئ", "ء", txt)
    txt = re.sub("ة", "ه", txt)
    txt = re.sub("گ", "ك", txt)
    print('remove longation', txt)
    
    # remove non-arabic words, or non-numbers, or non-english words in the text
    txt = re.sub(r'[^a-zA-Z\s0-9\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD.0-9]+'
                 ,' ', txt)
    print('remove non-arabic/non-english/non-number words', txt)
    
    return txt

NameError: name 'pd' is not defined

In [8]:
query_test = 'ي1'
query_test_cleaned = text_clean2_steps(query_test)
query_test_cleaned

NameError: name 'text_clean2_steps' is not defined

## 5- Calculating Similarities

In [209]:
display(text_clean_enc_df)
temp_x = text_clean_enc_df.values
display(temp_x)
display(text_clean_enc_df.loc[:, 0])
display(np.shape(temp_x[:, 0]))

q_temp = [q1_cleaned]
q_temp = clean2_vect.transform(q_temp).toarray()
q_temp = np.tile(np.array(q_temp).transpose(), (1, text_clean_enc_df.shape[1]))
q_temp = q_temp.reshape(-1, text_clean_enc_df.shape[0])
# print(q_temp)
# np.dot(text_clean_enc_df, q_temp)
# tile(array([[1,2,3]]).transpose(), (1, 3))

,0,1,2,3,4,5,6,7,8,9,...,5074,5075,5076,5077,5078,5079,5080,5081,5082,5083
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ہر,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ہمارے,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ہولوکاسٹ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ہےکہ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

00          0.0
000         0.0
0000        0.0
0000015     0.0
001         0.0
           ... 
ہر          0.0
ہمارے       0.0
ہولوکاسٹ    0.0
ہےکہ        0.0
یقول        0.0
Name: 0, Length: 23737, dtype: float64

(23737,)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [192]:
dict_t = {'dd': 1, 'fd': 2}
dict_t.items()

dict_items([('dd', 1), ('fd', 2)])

In [252]:
import numba

In [2]:
import numpy as np

In [6]:
np.linalg.norm([0, 1, 1]) * np.linalg.norm([0, 7, 0])

9.899494936611665

In [7]:
np.linalg.norm([0, 1, 1]) * np.linalg.norm([0, 1, 1])

2.0000000000000004

In [379]:
# optimized 1
from numba import njit
import numba

def get_similar_articles(q, df, vectorizer):
    # Convert the query become a vector
    q = [q]
    q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    df = df.values
    
    # Calculate the similarity
    sim = list(range(df.shape[1]))
    
    @njit
    def calc_sim(val):
        x = val
        num = np.linalg.norm(df[:, val]) * np.linalg.norm(q_vec)
        if num == 0:
            y = 0
        else:
            y = np.dot(df[:, val], q_vec) / num
        return x, y 
    
    # prepare result
    sim = list(map(calc_sim, sim))
    sim_sorted = sim
    return sim_sorted

In [36]:
# optimized 2
from numba import njit, prange

@njit(parallel=True, cache=True)
def compute_calc(np_data, q_vec, sim, n):
    for i in prange(n):
        num = np.linalg.norm(np_data[:, i]) * np.linalg.norm(q_vec)
        if num == 0:
            sim[i] = 0
        else:
            sim[i] = np.dot(np_data[:, i], q_vec) / num
    return sim

def get_similar_articles(q, df, vectorizer):
    # Convert the query become a vector
    q = [q]
    q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    
    # Calculate the similarity
    np_data=df.values
    sim = np.zeros((text_clean_enc_df.shape[1])) 
    range_n = np.array(range(np.shape(np_data)[1]))
    sim = compute_calc(np_data, q_vec, sim, np.shape(np_data)[1])
    
    
    # prepare result
    sim = list(enumerate(sim))
    sim_sorted = sorted(sim, key=lambda x: x[1], reverse=True)[:10]
    return sim_sorted


In [314]:
# import numpy as np
# from scipy.sparse import csr_matrix
# # !pip install sparse_dot_topn 
# import sparse_dot_topn.sparse_dot_topn as ct

# def awesome_cossim_top(A, B, ntop, lower_bound=0):
#     # force A and B as a CSR matrix.
#     # If they have already been CSR, there is no overhead
#     A = A.tocsr()
#     B = B.tocsr()
#     M, _ = A.shape
#     _, N = B.shape
 
#     idx_dtype = np.int32
 
#     nnz_max = M*ntop
 
#     indptr = np.zeros(M+1, dtype=idx_dtype)
#     indices = np.zeros(nnz_max, dtype=idx_dtype)
#     data = np.zeros(nnz_max, dtype=A.dtype)
#     ct.sparse_dot_topn(
#             M, N, np.asarray(A.indptr, dtype=idx_dtype),
#             np.asarray(A.indices, dtype=idx_dtype),
#             A.data,
#             np.asarray(B.indptr, dtype=idx_dtype),
#             np.asarray(B.indices, dtype=idx_dtype),
#             B.data,
#             ntop,
#             lower_bound,
#             indptr, indices, data)

#     return csr_matrix((data,indices,indptr),shape=(M,N))

In [358]:
start_time = time.time()
print((time.time() - start_time) * 10**3)

0.0


In [39]:
# Add The Query
q1 = 'مولد النبي'
q1_cleaned = text_clean2(q1)

# Measures
time_measure = None
most_freq_measure = None  

# q_preprocessed = vectorizer.fit_transform([q1]).transpose()
# text_clean_enc_df_2 = vectorizer.fit_transform(docs_df_cleaned['text_clean'])
start_time = time.time()
sorted_docs_with_scores = get_similar_articles(q1_cleaned, text_clean_enc_df, vectorizer=clean2_vect)  # call function
# awesome_cossim_top(text_clean_enc_df, q_preprocessed, 3, lower_bound=0)
time_measure = (time.time() - start_time) * 10**3

print('matching time taken', time_measure)

print('result', sorted(sorted_docs_with_scores, key=lambda x: x[1], reverse=True)[:10])

print()
print('correct result', [(4600, 0.20582365181966394),
 (2608, 0.20073743494298743),
 (2148, 0.2000789504882992),
 (1697, 0.19007115734405167),
 (3739, 0.16069003395532222),
 (2152, 0.1579702725541781),
 (4734, 0.15792581158425886),
 (2202, 0.1468077541726054),
 (546, 0.13791008813162506),
 (565, 0.13109829367759931)])

matching time taken 54.99911308288574
result [(4600, 0.20582365181966394), (2608, 0.20073743494298746), (2148, 0.2000789504882992), (1697, 0.19007115734405167), (3739, 0.16069003395532222), (2152, 0.1579702725541781), (4734, 0.15792581158425886), (2202, 0.1468077541726054), (546, 0.13791008813162506), (565, 0.1310982936775993)]

correct result [(4600, 0.20582365181966394), (2608, 0.20073743494298743), (2148, 0.2000789504882992), (1697, 0.19007115734405167), (3739, 0.16069003395532222), (2152, 0.1579702725541781), (4734, 0.15792581158425886), (2202, 0.1468077541726054), (546, 0.13791008813162506), (565, 0.13109829367759931)]


In [524]:
vocab_ = vectorizer.vocabulary_
print(f"number of unique words: {len(vocab_.keys())}")
most_freq_word = sorted(vocab_.items(), key=lambda x: x[1], reverse=True)[:1][0]
print('most frequent word is --> {} ({} times)'.format(most_freq_word[0], most_freq_word[1]))
score = len(vocab_.keys()) / most_freq_word[1]
print('Ratio: {:.3f}'.format(score))

most_freq_measure = most_freq_word[1]

number of unique words: 89018
most frequent word is --> یقول (89017 times)
Ratio: 1.000


In [436]:
docs_df_cleaned2

,tags,article_type,text_clean
0,"[التربية والتعليم, وزارة التربية والتعليم]",News,اكد امين وزاره تربيه تعليم شان مالي اداري دكتو...
1,"[مكرمة أبناء المعلمين, وزارة التربية والتعليم]",News,احتفل وزاره تربيه تعليم يوم معلم تكريم نخبه مع...
2,"[الغاز الروسي, أوروبا, أزمة الطاقة]",News,شهد عالم اول ازمه طاقه عالم حقيقي تاريخ . تنام...
3,"[التربية والتعليم, التوقيت الصيفي, المدارس]",News,كشف امين وزاره تربيه تعليم شان مالي اداري دكتو...
4,"[وزارة العمل, المولد النبوي]",News,اكد وزاره عمل ان عطل رسمي ماجور عامل مءسسه قطا...
...,...,...,...
5079,[الديموقراطية],Blog,تحدث قوه مءثر ارض عربي اصبح سبخ مالح كان بيءه ...
5080,"[ألوان الطيف السبعة, الشذوذ الجنسي]",Blog,خلق كون بدع صور ، جعل سماء درجه ، ارض منزل ، م...
5081,"[الإسلام, الإسلام السياسي]",Blog,حال صراع صفر ي نظام اسلام سياسي تولد ربيع عربي...
5082,[تخفيضات],Blog,جدل ساد شارع اردني مجدد اثر تخفيض كبير سعر جها...


In [443]:
q = 'مولد النبي'
time_measure = None
most_freq_measure = None  
start_time = time.time()

doc_ids = list(docs_df_cleaned2['tags'].index)
q_list = np.array(q.split(' '))
sim_score = list(np.zeros(docs_df_cleaned2.shape[0]))

for i, tag in enumerate(docs_df_cleaned2['tags']):
    for str_tag in tag:
        q_list_map = np.vectorize(lambda x: x in str_tag)(q_list) 
        if True in q_list_map:
            sim_score[i] += 1

sim_non_sorted = list(zip(doc_ids, sim_score))
sorted_docs_with_scores_content = sim_non_sorted

time_measure = (time.time() - start_time) * 10**3
score = 0
print('Ratio: {:.3f}'.format(score))
print()
print('time measure:', time_measure)

Ratio: 0.000

time measure: 164.09802436828613


## 6- getting top documents

In [528]:
sorted_docs_with_scores = sorted(sorted_docs_with_scores, key=lambda x: x[1], reverse=True)
top_5_docs = np.array(sorted_docs_with_scores, dtype='int32')[:5, 0]
top_5_docs

array([1697, 2152, 2608, 2527, 2147])

In [62]:
# results 
print('time measure:', time_measure)
print('frequency measure:', most_freq_measure)
print('score %.3f' % score)

time measure: 648.0000019073486
frequency measure: 89123
score 1.000
